# NLP con Hugging Face

## Procesando los datos para NLP

### Descargando el dataset

In [1]:
%%capture
!pip3 install evaluate

Usaremos el dataset MRPC. Este es uno de los 10 datasets que componen el [benchmark (punto de referencia) GLUE](https://huggingface.co/datasets/glue). Se utiliza para medir el rendimiento de los modelos ML en 10 tareas de clasificación de texto diferentes.

En otras palabras, seleccionamos el subset `mrpc` del dataset `glue`:

In [2]:
from datasets import load_dataset

dataset = load_dataset('glue', 'mrpc')

Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

Así se ve un ejemplo. Notamos que `mrpc` está compuesto de dos oraciones y una etiqueta que indica si los dos enunciados son equivalentes.

In [3]:
example = dataset['train'][0]
example

{'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .',
 'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .',
 'label': 1,
 'idx': 0}

In [5]:
labels = dataset['train'].features['label']
labels

ClassLabel(names=['not_equivalent', 'equivalent'], id=None)

In [6]:
labels.int2str(example['label'])

'equivalent'

### Tokenizando

¿Recuerdas que con visión descargamos el feature extractor directamente del repositorio del modelo pre-entrenado que vamos a usar como base?

Podemos pensar en la función tokenizadora como el equivalente en el NLP.

Descargamos el tokenizador directamente del repo del modelo que usaremos.

In [7]:
from transformers import AutoTokenizer

repo_id = 'bert-base-uncased'

tokenizer = AutoTokenizer.from_pretrained(repo_id)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

c:\Users\zer0\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\zer0\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
c:\Users\zer0\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_do

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Para preprocesar el conjunto de datos necesitamos convertir el texto en números que el modelo pueda entender. Esto se hace con un tokenizador. 

Pasar de texto a números se conoce como codificación o encoding. El encoding se realiza en un proceso de dos pasos: la tokenización, seguida de la conversión a input ids. Por el momento nos basta saber que estamos traduciendo texto a números llamados como input ids. Estos estarán en el formato adecuado para alimentar nuestro modelo.

Podemos alimentar al tokenizador con una oración o una lista de oraciones, por lo que podemos tokenizar directamente todas las primeras oraciones y todas las segundas oraciones de cada par de esta manera:

In [8]:
tokenized_sentence1 = tokenizer(example['sentence1'])
tokenized_sentence1

{'input_ids': [101, 2572, 3217, 5831, 5496, 2010, 2567, 1010, 3183, 2002, 2170, 1000, 1996, 7409, 1000, 1010, 1997, 9969, 4487, 23809, 3436, 2010, 3350, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Necesitamos manejar los dos enunciados como un par y no separados. El tokenizador puede tomar un par de secuencias y prepararlas de la manera que espera nuestro modelo:

In [9]:
inputs = tokenizer(example['sentence1'], example['sentence2'], return_tensors='pt')
inputs

{'input_ids': tensor([[  101,  2572,  3217,  5831,  5496,  2010,  2567,  1010,  3183,  2002,
          2170,  1000,  1996,  7409,  1000,  1010,  1997,  9969,  4487, 23809,
          3436,  2010,  3350,  1012,   102,  7727,  2000,  2032,  2004,  2069,
          1000,  1996,  7409,  1000,  1010,  2572,  3217,  5831,  5496,  2010,
          2567,  1997,  9969,  4487, 23809,  3436,  2010,  3350,  1012,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1]])}

¿Qué significa cada uno de los valores que nos retorna el tokenizador?
- `input_ids` es la traducción de palabras a números.
- `attention_mask` es un tensor con la misma forma que `input_ids`, pero lleno de 0 y 1: los 1 indican que se debe atender a los tokens correspondientes y los 0 indican que no se deben atender. Es decir, deben ser ignorados por el modelo.
- `token_type_ids` dice al modelo qué parte de la entrada es la primera oración y cuál es la segunda oración.

El modelo espera que las entradas sean de la forma [CLS] oración 1 [SEP] oración 2 [SEP] cuando hay dos oraciones.

In [14]:
tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])

['[CLS]',
 'am',
 '##ro',
 '##zi',
 'accused',
 'his',
 'brother',
 ',',
 'whom',
 'he',
 'called',
 '"',
 'the',
 'witness',
 '"',
 ',',
 'of',
 'deliberately',
 'di',
 '##stor',
 '##ting',
 'his',
 'evidence',
 '.',
 '[SEP]',
 'referring',
 'to',
 'him',
 'as',
 'only',
 '"',
 'the',
 'witness',
 '"',
 ',',
 'am',
 '##ro',
 '##zi',
 'accused',
 'his',
 'brother',
 'of',
 'deliberately',
 'di',
 '##stor',
 '##ting',
 'his',
 'evidence',
 '.',
 '[SEP]']

Si seleccionamos otro modelo en el Hub no necesariamente tendremos `token_type_ids` en las entradas tokenizadas (por ejemplo, no se devuelven si usa un modelo `DistilBERT`). Solo se devuelven cuando el modelo sabrá qué hacer con ellas, porque los ha visto durante su preentrenamiento.

En general, no necesitamos preocuparnos por si hay o no `token_type_ids` en nuestras entradas tokenizadas, siempre que usemos el tokenizador correspondiente al modelo, todo estará bien ya que el tokenizador sabe qué proporcionar al modelo.

Por ejemplo, durante esta clase utilizaremos un modelo [`distilroberta-base`](https://huggingface.co/distilroberta-base) por su tamaño y efectividad. Pero no cuenta con `token_type_ids` y aún así nos regresa excelentes resultados.

En la organización del Platzi en el Hub puedes encontrar un [modelo BERT](https://huggingface.co/platzi/platzi-distilroberta-base-mrpc-glue-omar-espejel) afinado siguiendo el mismo proceso que usamos en esta clase.

In [15]:
repo_id = 'distilroberta-base'

tokenizer = AutoTokenizer.from_pretrained(repo_id)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

c:\Users\zer0\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\zer0\.cache\huggingface\hub\models--distilroberta-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
c:\Users\zer0\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_d

config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Creamos una función tokenizadora. Recibe un ejemplo y lo tokeniza.

In [16]:
def tokenize_function(examples):
    return tokenizer(examples['sentence1'], examples['sentence2'], truncation=True)

In [17]:
prepared_dataset = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

### Definiendo el data collator: Dynamic padding

Necesitamos que nuestros tensores tengan una forma rectangular. Es decir que tengan el mismo tamaño cada uno de los ejemplos. Sin embargo, los textos no necesariamente tienen el mismo tamaño. 

Para ello usamos el relleno o padding. El padding se asegura de que todas nuestras oraciones tengan la misma longitud al agregar una palabra especial llamada padding token a las oraciones con menos valores. Por ejemplo, si tenemos 10 oraciones con 10 palabras y 1 oración con 20 palabras, el relleno garantizará que todas las oraciones tengan 20 palabras.

Dejamos el argumento de `padding` del tokenizer vacío en nuestra función de tokenización por ahora. Esto se debe a que rellenar (hacer padding) todas las muestras hasta la longitud máxima del dataset no es eficiente, es mejor rellenar las muestras cuando estamos construyendo un batch, ya que entonces solo necesitamos rellenar hasta la longitud máxima en ese batch, y no la longitud máxima en todo el dataset. ¡Esto puede ahorrar mucho tiempo y potencia de procesamiento cuando las entradas tienen longitudes muy variables!

Usaremos un DataCollator para esto.

Rellenemos (hagamos padding) todos los ejemplos con la longitud del elemento más largo del batch. A esta técnica se le conoce como relleno dinámico o dynamic padding.

In [18]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

## Entrenamiento y evaluación

Definamos el resto de los argumentos necesarios para `Trainer`.

### Definiendo la métrica 

In [19]:
import evaluate
import numpy as np

def compute_metrics(p):
    metric = evaluate.load('glue', 'mrpc')
    logits, labels = p
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

### Configurando `Trainer`


In [20]:
from transformers import AutoModelForSequenceClassification

labels = dataset['train'].features['label'].names

model = AutoModelForSequenceClassification.from_pretrained(
    repo_id, 
    num_labels=len(labels),
    id2label={str(i): label for i, label in enumerate(labels)},
    label2id={label: str(i) for i, label in enumerate(labels)}
)

model.safetensors:   0%|          | 0.00/331M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='./distilroberta-base-mrpc',
    evaluation_strategy='steps',
    num_train_epochs=3,
    push_to_hub=True,
    load_best_model_at_end=True,
)

c:\Users\zer0\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [22]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=prepared_dataset['train'],
    eval_dataset=prepared_dataset['validation'],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

### Entrenamiento

In [23]:
train_results = trainer.train()
trainer.save_model()
trainer.log_metrics('train', train_results.metrics)
trainer.save_metrics('train', train_results.metrics)

  0%|          | 0/1377 [00:00<?, ?it/s]

### Evaluación

In [ ]:
metrics = trainer.evaluate(prepared_dataset['validation'])
trainer.log_metrics('eval', metrics)
trainer.save_metrics('eval', metrics)

### Compartimos en el Hub

In [ ]:
kwargs = {
    'finetuned_model': model.config._name_or_path,
    'task': 'text-classification',
    'dataset': ['glue', 'mrpc'],
    'tags': ['text-classification', 'glue', 'mrpc'],
    'notes': 'Fine-tuned on GLUE MRPC dataset',
}

trainer.push_to_hub(**kwargs)